# Crypto Clustering

In [2]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
df_market_data = pd.read_csv(
    Path("Desktop/csv/crypto_market_data.csv"),
    index_col="coin_id")

df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [4]:
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [5]:
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

## Prepare the Data

In [6]:
scaled_data = StandardScaler().fit_transform(df_market_data)

In [7]:
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

df_market_data_scaled["coin_id"] = df_market_data.index

df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


## Find the Best Value for k using the Original Data

In [8]:
inertia = []
k = list(range(1, 11))

In [10]:
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)

In [11]:
elbow_data = {"k": k, "inertia": inertia}

df_elbow = pd.DataFrame(elbow_data)

df_elbow.head()

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.405923


In [12]:
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

### Question: What is the best value for k?
### Answer: k=1

## Cluster cryptocurrencies with K-means using the Original DataFrame

In [23]:
model = KMeans(n_clusters=1, random_state=0)

model.fit(df_market_data_scaled)

k_1 = model.predict(df_market_data_scaled)

df_market_data_scaled_predictions = df_market_data_scaled.copy()

df_market_data_scaled_predictions["customer_segments"] = k_1

In [24]:
df_market_data_scaled_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,customer_segments
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [25]:
df_market_data_scaled_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="customer_segments",
    hover_cols = ['Ticker'],
    title='Scatter Plot by Stock Segment - k=1'
)

:NdOverlay   [customer_segments]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

## Optimise Clusters with Principal Component Analysis

In [26]:
pca = PCA(n_components=2)

In [28]:
pca_data = pca.fit_transform(df_market_data_scaled_predictions)

pca_data[:5]

array([[-0.60066733,  0.84276006],
       [-0.45826071,  0.45846566],
       [-0.43306981, -0.16812638],
       [-0.47183495, -0.22266008],
       [-1.15779997,  2.04120919]])

In [29]:
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813])

### Question: What is the total explained variance of the three principal components
### Answer: 

In [30]:
df_crypto_pca = pd.DataFrame(pca_data, columns=['PC1', 'PC2'])
    
df_crypto_pca['Ticker'] = df_market_data_scaled.index

df_crypto_pca = df_crypto_pca.set_index('Ticker')

df_crypto_pca.head()

,PC1,PC2
Ticker,,
bitcoin,-0.600667,0.842760
ethereum,-0.458261,0.458466
tether,-0.433070,-0.168126
ripple,-0.471835,-0.222660
bitcoin-cash,-1.157800,2.041209


## Find the Best Value for k using the PCA Data

In [32]:
inertia = []
k = list(range(1, 11))

In [33]:
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_crypto_pca)
    inertia.append(k_model.inertia_)

In [34]:
elbow_data = {"k": k, "inertia": inertia}

df_elbow = pd.DataFrame(elbow_data)

df_elbow.head()

,k,inertia
0,1,206.351201
1,2,116.359216
2,3,52.498746
3,4,31.807943
4,5,22.079039


In [35]:
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

### Question: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

### Answer:

### Question: Does it differ from the best k value found using the original data?

### Answer: 

## Cluster Cryptocurrencies with K-means Using the PCA Data

In [36]:
model = KMeans(n_clusters=3)

model.fit(df_crypto_pca)

stock_clusters = model.predict(df_crypto_pca)

print(stock_clusters)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 2
 0 0 0 0]


In [38]:
df_pca_predictions = df_crypto_pca.copy()

df_pca_predictions['StockCluster'] = stock_clusters

df_pca_predictions

,PC1,PC2,StockCluster
Ticker,,,
bitcoin,-0.600667,0.842760,0
ethereum,-0.458261,0.458466,0
tether,-0.433070,-0.168126,0
ripple,-0.471835,-0.222660,0
bitcoin-cash,-1.157800,2.041209,0
binancecoin,-0.516534,1.388377,0
chainlink,-0.450711,0.517699,0
cardano,-0.345600,0.729439,0
litecoin,-0.649468,0.432165,0


In [42]:
df_pca_predictions.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='StockCluster',
    hover_cols = ['Ticker'],
    title='Scatter Plot by Stock Segment'
)

:NdOverlay   [StockCluster]
   :Scatter   [PC1]   (PC2,Ticker)

## Visualise and Compare the Results

### Question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

### Answer: